In [1]:
using Pkg

In [2]:
using TSSOS
using DynamicPolynomials
using Random
using Statistics
using DataFrames
using DelimitedFiles

In [3]:
function observation(T,ts,start)
    # randomly select a trajectory of time series
    return copy(ts[start:start+T-1])
end

observation (generic function with 1 method)

In [4]:
function parameter_estimation2(Y, order)
    # training process
    
    T=length(Y)
    
    @polyvar G[1:4] Fdash[1:2] p[1:T] q[1:2*T] f[1:T] m[1:2*(T+1)];
    var=vcat(G,Fdash,m,f,p,q);

    # constraints
    ine1 = [f[i] - Fdash[1]*m[2*i+1] - Fdash[2]*m[2*i+2] - p[i] for i in 1:T];
    ine2 = [-f[i] + Fdash[1]*m[2*i+1] + Fdash[2]*m[2*i+2] + p[i] for i in 1:T];
    ine3 = [m[i+2] - G[1]*m[i] - G[2]*m[i+1] - q[i]  for i in 1:2:2*T];
    ine4 = [-m[i+2] + G[1]*m[i] + G[2]*m[i+1] + q[i]  for i in 1:2:2*T];
    ine5 = [m[i+2] - G[3]*m[i-1] - G[4]*m[i] - q[i]  for i in 2:2:2*T];
    ine6 = [-m[i+2] + G[3]*m[i-1] + G[4]*m[i] + q[i]  for i in 2:2:2*T];

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T)+ 0.005*sum(p[i]^2 for i in 1:T) + 0.001*sum(q[i]^2 for i in 1:2*T)

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4,ine5,ine6);

    # solve model 
    opt,sol,data=tssos_first(pop,var,order,TS="MD",solution=true);
    
    println("sol: ", sol)
    #println("opt: ", opt)
    #println("data: ", data)
    return opt, sol, data 
end

parameter_estimation2 (generic function with 1 method)

In [5]:
function rmse(Y_predict, Y_true)
    # compare predicted Y_[T+1：T+pred] with actual value of predicted Y_[T+1：T+pred]
    # calcluate rmse: sqrt( sum((Y-f)^2)/N) 
    Y=copy(Y_true[1:length(Y_predict)])
    return  abs(Y[1]-Y_predict[1])
    #1-sum( abs(Y[i]-Y_predict[i]) for i in 1:length(Y_predict) )/ sum(abs(Y[i]-mean(Y)) for i in 1:length(Y_predict) )
end
    

rmse (generic function with 1 method)

In [7]:
#read Hazan data with process noise variance 0.1 and observation noise variance 0.2
ts = readdlm( "Hazan_25_ts.txt" );

In [ ]:
## a single run (because the first time using TSSOS is very slow)
pred=1
for T in 2:5
    for order in 1:3
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts,1)
        parameter_estimation2(Y[1:T], order)
    end
end
#T=4 # the length of time window is 20
#Y=observation(T+pred,ts,1) # select a 21-period time series from stock-market data. 
#opt, sol, data = parameter_estimation2(Y[1:T]) # system identification using the first 20-period

T=2, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2]
[8, 8]
-----------------------------------------------------------------------------
Obtained the block structure in 3.024340072 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 231 affine constraints.
SDP assembling time: 3.792392407 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 231             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 13              
  Matrix variables       : 32              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Free

4   2.4e+01  1.1e-01  6.8e-02  2.00e+00   4.928403314e+00   4.638322796e+00   1.1e-01  0.29  
5   6.2e+00  2.8e-02  8.0e-03  1.01e+00   4.763567332e+00   4.708402222e+00   2.8e-02  0.34  
6   1.2e+00  5.3e-03  6.5e-04  1.16e+00   2.953960130e+00   2.943455805e+00   5.3e-03  0.38  
7   7.8e-01  3.5e-03  4.1e-04  1.48e-01   2.342368034e+00   2.332330785e+00   3.5e-03  0.42  
8   6.7e-01  3.1e-03  3.2e-04  6.02e-01   1.930272981e+00   1.922539588e+00   3.1e-03  0.46  
9   2.6e-01  1.2e-03  8.4e-05  5.02e-01   7.878628299e-01   7.843383254e-01   1.2e-03  0.51  
10  3.3e-02  1.5e-04  2.8e-06  7.66e-01   1.588309950e-01   1.586976795e-01   1.5e-04  0.55  
11  6.8e-03  3.1e-05  2.5e-07  9.82e-01   4.301183047e-02   4.299120266e-02   3.1e-05  0.59  
12  3.6e-03  1.6e-05  8.4e-08  1.29e+00   2.151198368e-02   2.150548255e-02   1.6e-05  0.63  
13  1.1e-03  4.8e-06  1.1e-08  1.34e+00   6.294639603e-03   6.294282592e-03   4.8e-06  0.68  
14  4.4e-04  2.0e-06  2.8e-09  1.35e+00   2.294276534e-03   

2   1.9e+00  7.1e-02  3.4e-02  6.16e-01   4.950590507e+00   4.788520825e+00   7.1e-02  0.05  
3   3.2e-01  1.2e-02  4.5e-03  3.70e-01   2.081825351e+00   1.961347738e+00   1.2e-02  0.06  
4   5.7e-02  2.1e-03  3.6e-04  7.76e-01   5.957165932e-01   5.712820064e-01   2.1e-03  0.07  
5   4.8e-03  1.8e-04  3.8e-06  8.69e-01   5.609095168e-02   5.598023906e-02   1.8e-04  0.09  
6   8.8e-04  3.3e-05  1.7e-07  1.76e+00   7.757003267e-03   7.766566990e-03   3.3e-05  0.10  
7   2.7e-05  1.0e-06  8.1e-10  1.56e+00   1.898840114e-04   1.901072989e-04   1.0e-06  0.11  
8   2.9e-07  1.1e-08  8.7e-13  1.02e+00   2.019373834e-06   2.021579607e-06   1.1e-08  0.12  
9   1.6e-09  1.3e-10  3.6e-16  1.00e+00   1.128010750e-08   1.129207289e-08   5.9e-11  0.14  
Optimizer terminated. Time: 0.14    

SDP solving time: 0.150925803 seconds.
optimum = 1.128010749812156e-8
Found a locally optimal solution by Ipopt, giving an upper bound: 0.028636711242547484 and a relative optimality gap: 2.8636699962439986%.
s

Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 5130            
Presolve terminated. Time: 0.02    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 19088           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 6328            
  Matrix variables       : 5163            
  Integer variables      : 0               

Optimizer  - thread

4   1.0e-01  3.2e-03  6.6e-04  8.46e-01   1.165966059e+00   1.129447078e+00   3.2e-03  0.18  
5   1.2e-02  3.7e-04  1.7e-05  7.35e-01   1.496619787e-01   1.483109993e-01   3.7e-04  0.21  
6   2.2e-03  6.8e-05  6.8e-07  1.42e+00   2.898261073e-02   2.898076701e-02   6.8e-05  0.25  
7   3.9e-04  1.2e-05  3.6e-08  1.88e+00   3.082020660e-03   3.084259753e-03   1.2e-05  0.29  
8   5.9e-06  1.8e-07  6.1e-11  1.24e+00   4.226971014e-05   4.230321568e-05   1.8e-07  0.32  
9   1.0e-07  3.2e-09  1.4e-13  1.00e+00   7.358355487e-07   7.364029774e-07   3.1e-09  0.35  
Optimizer terminated. Time: 0.35    

SDP solving time: 0.372596036 seconds.
optimum = 7.358355486571317e-7
The local solver failed!
sol: [1.804699738344413e-15, 1.1730548299238683e-15, -5.363968666745894e-16, -1.2582767620123546e-15, 4.724804176082248e-16, -3.56240208766944e-16, 5.414099215033238e-16, 3.007832897240688e-17, 3.684595299119843e-16, -1.0778067881779133e-16, 6.855352478294402e-16, -2.0302872056374646e-16, 5.91540469790

Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.03            
Lin. dep.  - number                 : 11700           
Presolve terminated. Time: 0.05    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 36401           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 13233           
  Matrix variables       : 9152            
  Integer variables      : 0               

Optimizer  - thread

2   3.5e+00  7.4e-02  4.5e-02  1.24e+00   8.970012950e+00   8.662142630e+00   7.4e-02  0.34  
3   8.4e-01  1.8e-02  8.3e-03  4.79e-01   4.686931799e+00   4.495037366e+00   1.8e-02  0.40  
4   2.0e-01  4.3e-03  1.1e-03  9.44e-01   2.042285958e+00   1.987193663e+00   4.3e-03  0.48  
5   3.9e-02  8.3e-04  8.6e-05  5.88e-01   4.915058708e-01   4.823628755e-01   8.3e-04  0.54  
6   5.2e-03  1.1e-04  3.1e-06  1.15e+00   7.306524721e-02   7.245580655e-02   1.1e-04  0.61  
7   8.4e-04  1.8e-05  1.3e-07  1.82e+00   8.373963189e-03   8.340259652e-03   1.8e-05  0.67  
8   1.9e-05  4.1e-07  4.5e-10  1.54e+00   1.279109617e-04   1.270022187e-04   4.1e-07  0.75  
9   1.9e-07  4.0e-09  4.4e-13  1.00e+00   1.298671815e-06   1.289638026e-06   4.0e-09  0.81  
10  2.2e-09  5.9e-10  7.9e-16  9.99e-01   1.967487345e-08   1.954047959e-08   6.0e-11  0.89  
Optimizer terminated. Time: 0.90    

SDP solving time: 0.972201036 seconds.
optimum = 1.9674873452440315e-8
Found a locally optimal solution by Ipopt, gi

Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[39, 31, 6, 2, 1]
[39, 1, 6, 30, 9109]
-----------------------------------------------------------------------------
Obtained the block structure in 203.653433036 seconds. The maximal size of blocks is 39.
Assembling the SDP...
There are 61742 affine constraints.
SDP assembling time: 30.912100581 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 61742           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 23990           
  Matrix variables       : 14653           
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checke

In [ ]:
#check for T=10 only
f5 = sol[5]*sol[17] + sol[6]*sol[18] + sol[43]
f5real = sol[33]
println(f5)
println(f5real)

In [ ]:
pred=1
for T in 5:10
    for order in 1:2
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts,1)
        parameter_estimation2(Y[1:T], order)
    end
end

T=5, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[8, 4, 20]
-----------------------------------------------------------------------------
Obtained the block structure in 3.091470648 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 780 affine constraints.
SDP assembling time: 3.792762496 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 780             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 64              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator starte

2   6.4e+02  8.3e-01  2.8e+00  1.50e+00   1.723952071e+01   9.117755921e+00   8.3e-01  5.12  
3   6.1e+02  7.8e-01  2.4e+00  6.61e+00   1.510499438e+01   7.827444574e+00   7.8e-01  6.16  
4   4.5e+02  5.9e-01  9.9e-01  4.77e+00   7.898194538e+00   5.634301349e+00   5.9e-01  7.23  
5   5.7e+01  7.3e-02  8.7e-02  1.60e+00   1.433949758e+01   1.302209925e+01   7.3e-02  8.25  
6   2.2e+01  2.8e-02  1.8e-02  1.24e+00   1.288933921e+01   1.249604955e+01   2.8e-02  9.36  
7   6.0e+00  7.8e-03  9.7e-04  1.44e+00   1.007021574e+01   1.005985106e+01   7.8e-03  10.38 
8   1.9e+00  2.5e-03  2.6e-04  9.07e-01   6.576309346e+00   6.567857122e+00   2.5e-03  11.46 
9   1.1e+00  1.4e-03  1.1e-04  7.94e-01   4.890770823e+00   4.885915135e+00   1.4e-03  12.49 
10  5.5e-01  7.1e-04  4.1e-05  7.24e-01   3.387630288e+00   3.385004299e+00   7.1e-04  13.54 
11  3.7e-01  4.8e-04  2.3e-05  8.16e-01   2.490289750e+00   2.488651448e+00   4.8e-04  14.60 
12  8.7e-02  1.1e-04  2.7e-06  7.05e-01   9.522117839e-01   

-----------------------------------------------------------------------------
The sizes of PSD blocks:
[45, 5, 3, 2, 1]
[1, 10, 4, 60, 910]
-----------------------------------------------------------------------------
Obtained the block structure in 1.247058226 seconds. The maximal size of blocks is 45.
Assembling the SDP...
There are 5339 affine constraints.
SDP assembling time: 3.148430165 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 5339            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2235            
  Matrix variables       : 1268            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.


2   5.8e+00  8.8e-02  5.9e-02  1.81e+00   1.358072298e+01   1.319969574e+01   8.8e-02  0.89  
3   1.6e+00  2.5e-02  1.2e-02  7.65e-01   7.722751099e+00   7.534057869e+00   2.5e-02  1.10  
4   4.6e-01  7.0e-03  1.9e-03  1.08e+00   4.247381572e+00   4.181690009e+00   7.0e-03  1.33  
5   1.2e-01  1.8e-03  3.1e-04  3.85e-01   1.643461355e+00   1.617064241e+00   1.8e-03  1.56  
6   1.7e-02  2.6e-04  1.5e-05  1.03e+00   2.951531037e-01   2.920885378e-01   2.6e-04  1.78  
7   3.0e-03  4.5e-05  6.5e-07  1.29e+00   4.718555015e-02   4.703529293e-02   4.5e-05  2.00  
8   5.3e-04  8.0e-06  2.6e-08  1.93e+00   4.779389577e-03   4.774949040e-03   8.0e-06  2.22  
9   2.2e-06  3.4e-08  6.9e-12  1.25e+00   1.790973808e-05   1.788935322e-05   3.4e-08  2.43  
10  1.7e-08  5.2e-10  4.8e-15  1.00e+00   1.428381531e-07   1.426778601e-07   2.6e-10  3.26  
Optimizer terminated. Time: 3.26    

SDP solving time: 3.396766469 seconds.
optimum = 1.4283815312851716e-7
Found a locally optimal solution by Ipopt, gi

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[14, 4, 32]
-----------------------------------------------------------------------------
Obtained the block structure in 0.020341935 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 1653 affine constraints.
SDP assembling time: 0.786251227 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1653            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 49              
  Matrix variables     

2   1.3e+03  7.8e-01  2.3e+00  5.42e-01   2.126808740e+01   1.460835883e+01   7.8e-01  1048.26
3   8.3e+02  5.0e-01  6.4e-01  6.77e+00   8.563737106e+00   7.277979749e+00   5.0e-01  1584.46
4   5.2e+01  3.1e-02  1.5e-02  4.64e-01   2.036715005e+01   2.016490500e+01   3.1e-02  2004.76
5   1.8e+01  1.1e-02  2.5e-03  1.20e+00   1.850710258e+01   1.846738993e+01   1.1e-02  2446.51
6   6.5e+00  3.9e-03  1.5e-04  1.79e+00   1.322833432e+01   1.322884577e+01   3.9e-03  3053.66
7   2.3e+00  1.4e-03  5.7e-05  4.42e-01   9.561584050e+00   9.561255309e+00   1.4e-03  3529.74
8   7.4e-01  4.5e-04  1.0e-05  1.14e+00   6.135749883e+00   6.135644248e+00   4.5e-04  3949.66
9   4.6e-01  2.8e-04  5.9e-06  4.21e-01   4.876477362e+00   4.876335375e+00   2.8e-04  4366.70


In [ ]:
pred=1
for T in 11:20
    println("T=",T)
    Y=observation(T+pred,ts,1)
    parameter_estimation2(Y[1:T], 1)
end

T=11
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[20, 4, 44]
-----------------------------------------------------------------------------
Obtained the block structure in 2.906974979 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 2850 affine constraints.
SDP assembling time: 4.163778802 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2850            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 67              
  Matrix variables       : 136             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator star

2   3.4e+01  2.9e-01  3.1e-01  4.18e+00   1.640867316e+01   1.552610278e+01   2.9e-01  8.21  
3   8.1e+00  7.0e-02  2.2e-02  1.07e+00   1.667110864e+01   1.661203244e+01   7.0e-02  10.23 
4   2.2e+00  1.9e-02  3.6e-03  5.72e-01   1.091716293e+01   1.089353486e+01   1.9e-02  12.29 
5   4.7e-01  4.1e-03  4.3e-04  5.32e-02   5.544338012e+00   5.537391397e+00   4.1e-03  14.40 
6   6.5e-02  5.7e-04  1.1e-05  3.40e-01   1.075237727e+00   1.075588232e+00   5.7e-04  16.46 
7   7.3e-03  6.3e-05  2.5e-07  1.07e+00   1.248416418e-01   1.248962130e-01   6.3e-05  18.49 
8   1.1e-03  9.7e-06  1.4e-08  1.86e+00   1.268315620e-02   1.268751019e-02   9.7e-06  20.51 
9   3.6e-05  3.1e-07  1.3e-10  1.52e+00   3.064470492e-04   3.064406245e-04   3.1e-07  22.45 
10  1.5e-06  1.3e-08  8.2e-13  1.01e+00   1.322553873e-05   1.322831575e-05   1.3e-08  24.45 
11  2.6e-09  1.3e-10  9.0e-18  1.00e+00   7.465940095e-09   7.467146982e-09   7.1e-12  26.48 
Optimizer terminated. Time: 26.49   

SDP solving time: 26.9

SDP assembling time: 1.578424138 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4371            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 85              
  Matrix variables       : 172             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00 

2   9.3e+01  6.3e-01  3.0e-01  6.89e+00   9.845811200e+00   1.020819399e+01   6.3e-01  54.05 
3   2.2e+01  1.5e-01  6.3e-02  1.60e+00   2.806412041e+01   2.803475979e+01   1.5e-01  60.17 
4   5.8e+00  3.9e-02  8.5e-03  2.67e+00   1.524108878e+01   1.521404094e+01   3.9e-02  66.05 
5   1.7e+00  1.1e-02  1.7e-03  6.12e-01   9.760112030e+00   9.745013208e+00   1.1e-02  71.75 
6   3.8e-01  2.6e-03  2.0e-04  3.26e-01   4.541192107e+00   4.537751107e+00   2.6e-03  77.32 
7   6.1e-02  4.2e-04  9.7e-06  6.22e-01   9.611497193e-01   9.611015101e-01   4.2e-04  82.73 
8   7.6e-03  5.2e-05  2.5e-07  1.11e+00   1.151924830e-01   1.152180881e-01   5.2e-05  88.37 
9   8.8e-04  6.0e-06  3.4e-09  2.02e+00   8.033551737e-03   8.036373666e-03   6.0e-06  93.86 
10  2.0e-05  1.3e-07  2.7e-11  1.42e+00   1.334683572e-04   1.334862259e-04   1.3e-07  99.13 
11  8.0e-09  5.9e-11  2.5e-16  1.00e+00   6.249656069e-08   6.250381890e-08   6.0e-11  104.56
Optimizer terminated. Time: 105.18  

SDP solving time: 106.

## Try with process noise variance 0.01 and observation noise variance 0.05

In [4]:
#read Hazan data with process noise variance 0.01 and observation noise variance 0.05
ts2 = readdlm( "Hazan_25_ts_o005_p001.txt" );

In [8]:
pred=1
for T in 2:4
    for order in 1:3
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts2,1)
        parameter_estimation2(Y[1:T], order)
    end
end

T=2, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2]
[8, 8]
-----------------------------------------------------------------------------
Obtained the block structure in 3.244244225 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 231 affine constraints.
SDP assembling time: 4.093614695 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 231             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 13              
  Matrix variables       : 32              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Free

4   9.7e+00  4.4e-02  1.7e-02  1.48e+00   4.598427466e+00   4.483493578e+00   4.4e-02  0.27  
5   2.7e+00  1.2e-02  2.0e-03  1.21e+00   3.423049839e+00   3.406507842e+00   1.2e-02  0.30  
6   1.4e+00  6.5e-03  9.1e-04  7.92e-01   2.457479643e+00   2.444243188e+00   6.5e-03  0.34  
7   8.9e-01  4.0e-03  4.0e-04  7.61e-01   1.509849470e+00   1.503948243e+00   4.0e-03  0.38  
8   3.5e-01  1.6e-03  1.0e-04  6.62e-01   7.503212153e-01   7.478595406e-01   1.6e-03  0.42  
9   7.7e-02  3.5e-04  1.2e-05  8.08e-01   2.164628045e-01   2.157644289e-01   3.5e-04  0.46  
10  1.1e-02  4.8e-05  6.1e-07  9.37e-01   4.597720848e-02   4.588041267e-02   4.8e-05  0.50  
11  3.4e-03  1.5e-05  8.5e-08  1.17e+00   1.745790075e-02   1.744540838e-02   1.5e-05  0.54  
12  1.4e-03  6.6e-06  1.9e-08  1.40e+00   6.319548367e-03   6.317334764e-03   6.6e-06  0.58  
13  4.1e-04  1.8e-06  2.3e-09  1.34e+00   1.587708098e-03   1.587592630e-03   1.8e-06  0.62  
14  2.6e-05  1.2e-07  3.0e-11  1.16e+00   9.597747582e-05   

SDP assembling time: 0.012220451 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 378             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 19              
  Matrix variables       : 40              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00 

12  6.5e-03  1.8e-05  9.3e-08  1.09e+00   4.494298128e-02   4.493628506e-02   1.8e-05  2.17  
13  4.2e-03  1.2e-05  4.5e-08  1.37e+00   2.666444237e-02   2.666182013e-02   1.2e-05  2.33  
14  2.1e-03  5.7e-06  1.4e-08  1.35e+00   1.242438060e-02   1.242379022e-02   5.7e-06  2.48  
15  7.8e-04  2.1e-06  2.6e-09  1.40e+00   3.817825964e-03   3.817919850e-03   2.1e-06  2.63  
16  1.2e-04  3.3e-07  1.5e-10  1.25e+00   5.577717991e-04   5.578076754e-04   3.3e-07  2.79  
17  6.7e-07  1.8e-09  5.4e-14  1.05e+00   3.278965823e-06   3.279273742e-06   1.8e-09  2.93  
18  2.7e-07  1.5e-11  2.4e-17  1.00e+00   2.211245394e-08   2.211419550e-08   1.1e-11  3.43  
19  2.7e-07  1.5e-11  2.4e-17  1.00e+00   2.211245394e-08   2.211419550e-08   1.1e-11  4.16  
Optimizer terminated. Time: 4.85    

SDP solving time: 5.173278403 seconds.
termination status: SLOW_PROGRESS
solution status: FEASIBLE_POINT
optimum = 2.211245393500377e-8
Global optimality certified with relative optimality gap 2.211245304680230

Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 561             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 25              
  Matrix variables       : 48              
  Integer variables      : 0               

Optimizer  - thread

18  4.6e-07  2.8e-11  9.2e-17  1.00e+00   8.631278210e-08   8.631645123e-08   2.7e-11  10.56 
Optimizer terminated. Time: 12.15   

SDP solving time: 12.944584643 seconds.
termination status: SLOW_PROGRESS
solution status: FEASIBLE_POINT
optimum = 8.631278209547154e-8
Global optimality certified with relative optimality gap 8.631278253914829e-6%!
sol: [0.9005935803418353, 0.39870382451219544, 0.049342248297034534, 0.4815952585658128, 4.841520535738016, 1.0881445337736348, 0.32402594004098106, -0.04838643287231112, 0.2725238256471827, -0.0073145083322250564, 0.2425168903339116, 0.009924303809646748, 0.22236599679805888, 0.016745826228106758, 0.2069380252651497, 0.019036751328600464, 1.3114704559106045, 1.1849495825427134, 1.0948114181573902, 1.0226094363047495, -1.7950001803032222e-10, 7.854201345072527e-10, -1.0642273397488625e-9, 4.5914648317079096e-10, -1.1166794271609946e-11, 2.3941993687225405e-11, 4.9253852274091886e-9, -2.0179912166532525e-9, -1.5631641005933848e-8, -1.4367430728

In [9]:
pred=1
for T in 5:7
    for order in 1:2
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts2,1)
        parameter_estimation2(Y[1:T], order)
    end
end

T=5, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[8, 4, 20]
-----------------------------------------------------------------------------
Obtained the block structure in 0.00174175 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 780 affine constraints.
SDP assembling time: 0.08448695 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 780             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix varia

4   3.3e+02  4.2e-01  3.9e-01  4.30e+00   5.337225826e+00   4.810732149e+00   4.2e-01  8.78  
5   1.7e+01  2.1e-02  6.4e-03  5.46e-01   1.091399957e+01   1.084421187e+01   2.1e-02  9.80  
6   8.3e+00  1.1e-02  1.7e-03  1.30e+00   8.851499070e+00   8.834869890e+00   1.1e-02  10.85 
7   1.9e+00  2.5e-03  1.4e-04  1.62e+00   4.809824745e+00   4.808183640e+00   2.5e-03  11.94 
8   9.5e-01  1.2e-03  6.4e-05  3.79e-01   3.554354355e+00   3.552704183e+00   1.2e-03  12.98 
9   2.5e-01  3.3e-04  8.9e-06  6.04e-01   1.614017750e+00   1.613631143e+00   3.3e-04  14.00 
10  5.5e-02  7.1e-05  1.0e-06  7.63e-01   5.090189042e-01   5.089003269e-01   7.1e-05  15.07 
11  1.4e-02  1.8e-05  1.2e-07  9.12e-01   1.402957725e-01   1.402745365e-01   1.8e-05  16.23 
12  5.9e-03  7.6e-06  2.5e-08  1.36e+00   5.317095534e-02   5.316835076e-02   7.6e-06  17.44 
13  3.8e-03  4.9e-06  1.1e-08  1.53e+00   2.924705634e-02   2.924620392e-02   4.9e-06  18.45 
14  1.0e-03  1.3e-06  1.2e-09  1.50e+00   6.703019665e-03   

-----------------------------------------------------------------------------
The sizes of PSD blocks:
[45, 5, 3, 2, 1]
[1, 10, 4, 60, 910]
-----------------------------------------------------------------------------
Obtained the block structure in 1.383801429 seconds. The maximal size of blocks is 45.
Assembling the SDP...
There are 5339 affine constraints.
SDP assembling time: 3.471095029 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 5339            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2235            
  Matrix variables       : 1268            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.


2   2.8e+01  4.0e-01  7.6e-01  3.92e+00   1.285547817e+01   9.754199112e+00   4.0e-01  2.07  
3   5.9e+00  8.4e-02  4.2e-02  1.91e+00   9.547746421e+00   9.355295448e+00   8.4e-02  2.27  
4   1.8e+00  2.6e-02  8.8e-03  9.48e-01   5.004913764e+00   4.916874647e+00   2.6e-02  2.47  
5   5.2e-01  7.5e-03  1.3e-03  1.15e+00   2.527448892e+00   2.503527101e+00   7.5e-03  2.67  
6   1.4e-01  2.0e-03  2.2e-04  3.99e-01   9.244679690e-01   9.147222125e-01   2.0e-03  2.92  
7   1.6e-02  2.3e-04  7.3e-06  1.06e+00   1.306585740e-01   1.298761003e-01   2.3e-04  3.13  
8   3.0e-03  4.2e-05  3.3e-07  1.53e+00   2.047415709e-02   2.044471583e-02   4.2e-05  3.34  
9   4.3e-04  6.2e-06  1.2e-08  1.78e+00   1.732225289e-03   1.731798620e-03   6.2e-06  3.57  
10  9.5e-07  1.3e-08  9.9e-13  1.12e+00   3.727013289e-06   3.728168318e-06   1.3e-08  3.79  
11  4.4e-09  1.9e-10  3.1e-16  1.00e+00   1.754367516e-08   1.754924117e-08   6.2e-11  4.06  
Optimizer terminated. Time: 4.06    

SDP solving time: 4.67

In [14]:
pred=1
for T in 8:15
    println("T=",T)
    Y=observation(T+pred,ts2,1)
    parameter_estimation2(Y[1:T], 1)
end

T=8
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[14, 4, 32]
-----------------------------------------------------------------------------
Obtained the block structure in 0.020465898 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 1653 affine constraints.
SDP assembling time: 0.425160247 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1653            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 49              
  Matrix variables 

2   3.8e+01  4.2e-01  1.8e-01  5.65e+00   7.641605911e+00   7.810122868e+00   4.2e-01  5.58  
3   7.6e+00  8.4e-02  8.3e-03  1.04e+00   1.044573024e+01   1.048705542e+01   8.4e-02  6.15  
4   1.9e+00  2.1e-02  1.4e-03  1.08e+00   5.523983089e+00   5.532158095e+00   2.1e-02  6.73  
5   4.1e-01  4.5e-03  2.1e-04  1.61e-01   2.331598857e+00   2.333322035e+00   4.5e-03  7.30  
6   5.2e-02  5.8e-04  1.1e-05  4.85e-01   3.415130124e-01   3.417231993e-01   5.8e-04  7.88  
7   6.3e-03  7.0e-05  2.7e-07  1.22e+00   4.051720741e-02   4.055355177e-02   7.0e-05  8.46  
8   7.1e-04  7.9e-06  4.4e-09  1.78e+00   3.310711443e-03   3.314056020e-03   7.9e-06  9.05  
9   6.9e-06  7.6e-08  1.0e-11  1.28e+00   2.478374620e-05   2.479631181e-05   7.6e-08  9.58  
10  5.9e-09  1.4e-10  2.4e-16  1.00e+00   2.028869489e-08   2.029961224e-08   6.2e-11  10.16 
Optimizer terminated. Time: 10.17   

SDP solving time: 10.404292734 seconds.
optimum = 2.0288694886609247e-8
Global optimality certified with relative op

SDP assembling time: 0.761125556 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2850            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 67              
  Matrix variables       : 136             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00 

2   8.3e+01  6.8e-01  3.2e-01  8.04e+00   6.396612327e+00   6.906961031e+00   6.8e-01  11.45 
3   1.7e+01  1.4e-01  7.5e-02  2.44e+00   1.856615627e+01   1.840429539e+01   1.4e-01  13.34 
4   4.4e+00  3.6e-02  8.5e-03  2.77e+00   8.032453319e+00   7.993762301e+00   3.6e-02  15.59 
5   1.2e+00  9.9e-03  1.5e-03  6.61e-01   4.426438295e+00   4.410273137e+00   9.9e-03  17.74 
6   2.1e-01  1.7e-03  8.6e-05  5.02e-01   1.439135681e+00   1.438108574e+00   1.7e-03  19.65 
7   3.7e-02  3.0e-04  5.4e-07  5.31e-01   1.961484753e-01   1.964036633e-01   3.0e-04  21.80 
8   6.9e-03  5.6e-05  5.9e-08  1.32e+00   4.448159125e-02   4.451869392e-02   5.6e-05  23.94 
9   9.4e-04  7.7e-06  7.2e-09  2.04e+00   3.785275319e-03   3.787378986e-03   7.7e-06  26.01 
10  1.8e-05  1.5e-07  2.3e-11  1.23e+00   5.944882318e-05   5.947787410e-05   1.5e-07  28.17 
11  1.9e-07  1.6e-09  2.4e-14  1.00e+00   6.434462561e-07   6.437434998e-07   1.6e-09  30.48 
Optimizer terminated. Time: 30.49   

SDP solving time: 30.9

Assembling the SDP...
There are 4371 affine constraints.
SDP assembling time: 1.568881717 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4371            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 85              
  Matrix variables       : 172             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number           

2   1.3e+02  8.1e-01  4.3e-01  1.02e+01   5.673748934e+00   6.538952241e+00   8.1e-01  22.31 
3   2.0e+01  1.3e-01  1.2e-01  4.59e+00   2.137692359e+01   2.064253464e+01   1.3e-01  27.64 
4   5.8e+00  3.8e-02  1.1e-02  2.83e+00   8.941541115e+00   8.869612183e+00   3.8e-02  33.04 
5   2.5e+00  1.6e-02  3.9e-03  7.46e-01   6.219199281e+00   6.169268882e+00   1.6e-02  37.96 
6   6.4e-01  4.2e-03  5.7e-04  1.20e+00   2.760261322e+00   2.743845737e+00   4.2e-03  43.09 
7   9.0e-02  5.8e-04  1.6e-05  5.37e-01   5.962152233e-01   5.959313928e-01   5.8e-04  48.17 
8   1.2e-02  7.6e-05  4.7e-07  1.17e+00   7.238820608e-02   7.239684211e-02   7.6e-05  53.27 
9   1.3e-03  8.4e-06  7.1e-09  1.97e+00   4.994137388e-03   4.996330626e-03   8.4e-06  58.75 
10  5.6e-05  3.6e-07  8.7e-11  1.39e+00   1.660453989e-04   1.660929480e-04   3.6e-07  63.78 
11  6.5e-09  4.4e-11  6.6e-17  1.01e+00   1.232072088e-08   1.232580002e-08   3.0e-11  68.95 
Optimizer terminated. Time: 69.00   

SDP solving time: 69.7

## Try with process noise variance 0.001 and observation noise variance 0.002

In [6]:
ts3 = readdlm( "Hazan_25_ts_o0002_p0001.txt" );

In [12]:
pred=1
for T in 2:4
    for order in 1:3
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts3,1)
        parameter_estimation2(Y[1:T], order)
    end
end

T=2, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2]
[8, 8]
-----------------------------------------------------------------------------
Obtained the block structure in 0.003520213 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 231 affine constraints.
SDP assembling time: 0.03539648 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 231             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 13              
  Matrix variables  

4   8.7e+00  3.9e-02  1.5e-02  1.37e+00   4.513942016e+00   4.414808790e+00   3.9e-02  0.33  
5   1.9e+00  8.8e-03  1.1e-03  1.24e+00   3.103417049e+00   3.095533590e+00   8.8e-03  0.37  
6   1.3e+00  5.7e-03  6.6e-04  4.85e-01   2.451770068e+00   2.444199848e+00   5.7e-03  0.41  
7   7.8e-01  3.5e-03  2.8e-04  7.03e-01   1.373498015e+00   1.370463990e+00   3.5e-03  0.45  
8   3.1e-01  1.4e-03  8.1e-05  6.27e-01   6.749779606e-01   6.733955306e-01   1.4e-03  0.49  
9   4.5e-02  2.1e-04  5.0e-06  7.99e-01   1.380129105e-01   1.376885293e-01   2.1e-04  0.54  
10  8.6e-03  3.9e-05  3.9e-07  9.96e-01   3.734636590e-02   3.729525419e-02   3.9e-05  0.58  
11  3.7e-03  1.7e-05  9.3e-08  1.26e+00   1.684631168e-02   1.683481521e-02   1.7e-05  0.62  
12  1.1e-03  5.0e-06  1.1e-08  1.35e+00   4.772276269e-03   4.771564930e-03   5.0e-06  0.66  
13  4.3e-04  1.9e-06  2.5e-09  1.35e+00   1.618124691e-03   1.618040051e-03   1.9e-06  0.70  
14  3.7e-05  1.7e-07  5.6e-11  1.16e+00   1.319732546e-04   

2   1.7e+00  6.0e-02  2.9e-02  8.13e-01   3.638926833e+00   3.453219779e+00   6.0e-02  0.06  
3   3.3e-01  1.2e-02  4.1e-03  3.92e-01   1.422055093e+00   1.314765918e+00   1.2e-02  0.08  
4   5.1e-02  1.8e-03  2.6e-04  8.77e-01   3.344809453e-01   3.168401758e-01   1.8e-03  0.09  
5   3.7e-03  1.3e-04  2.5e-06  9.79e-01   3.058072477e-02   3.039211323e-02   1.3e-04  0.10  
6   7.7e-04  2.7e-05  1.0e-07  1.94e+00   3.262578952e-03   3.271186397e-03   2.7e-05  0.11  
7   1.2e-05  4.1e-07  1.2e-10  1.37e+00   4.649341440e-05   4.667596032e-05   4.1e-07  0.12  
8   1.4e-07  5.0e-09  1.7e-13  1.01e+00   5.840689069e-07   5.862967924e-07   5.0e-09  0.13  
9   1.8e-09  1.1e-09  3.8e-16  1.00e+00   1.085678969e-08   1.089129435e-08   8.6e-11  0.15  
Optimizer terminated. Time: 0.15    

SDP solving time: 0.160733061 seconds.
optimum = 1.0856789694867998e-8
Global optimality certified with relative optimality gap 1.0856789916898725e-6%!
sol: [0.41837461634065914, 0.2573493538543942, 0.481110518

Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.04            
Lin. dep.  - number                 : 5130            
Presolve terminated. Time: 0.05    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 19088           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 6328            
  Matrix variables       : 5163            
  Integer variables      : 0               

Optimizer  - thread

6   2.1e-03  6.2e-05  4.0e-07  1.63e+00   1.513448549e-02   1.515323325e-02   6.2e-05  0.23  
7   3.6e-04  1.0e-05  2.0e-08  1.71e+00   1.435077227e-03   1.438330093e-03   1.0e-05  0.25  
8   1.4e-06  4.0e-08  3.6e-12  1.15e+00   5.522086279e-06   5.538644169e-06   4.0e-08  0.28  
9   5.7e-08  1.7e-09  3.1e-14  1.00e+00   2.402056187e-07   2.408999787e-07   1.7e-09  0.30  
Optimizer terminated. Time: 0.30    

SDP solving time: 0.324591558 seconds.
optimum = 2.402056186721686e-7
Found a locally optimal solution by Ipopt, giving an upper bound: 0.025528167430052137 and a relative optimality gap: 2.552792722443346%.
sol: [3.16642528890333e-15, -5.2807333250847666e-15, 4.687722904455395e-16, 1.0520566772547163e-15, 3.0215009139289e-9, 7.691691233191553e-9, 1.7181174374058887e-16, 1.151138650007422e-15, 1.1003425850469985e-9, 2.5370887111356687e-9, 7.686738925711521e-10, 2.4978628937276553e-9, 7.975165964142245e-10, 2.480222583912006e-9, 2.5331471345299786e-10, 1.3403817154678841e-9, 1.291

Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.05            
Lin. dep.  - number                 : 11700           
Presolve terminated. Time: 0.07    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 36401           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 13233           
  Matrix variables       : 9152            
  Integer variables      : 0               

Optimizer  - thread

In [13]:
pred=1
for T in 5:7
    for order in 1:2
        println("T=",T, ", order=", order)
        Y=observation(T+pred,ts3,1)
        parameter_estimation2(Y[1:T], order)
    end
end

T=5, order=1
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[8, 4, 20]
-----------------------------------------------------------------------------
Obtained the block structure in 0.006271532 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 780 affine constraints.
SDP assembling time: 0.121873668 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 780             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix var

2   6.6e+02  8.5e-01  3.1e+00  3.21e+00   1.663936717e+01   7.179270574e+00   8.5e-01  4.84  
3   6.2e+02  8.0e-01  2.5e+00  6.65e+00   1.353621310e+01   6.048677849e+00   8.0e-01  5.79  
4   3.2e+02  4.1e-01  3.6e-01  4.16e+00   5.286494167e+00   4.826760752e+00   4.1e-01  6.76  
5   1.5e+01  1.9e-02  5.4e-03  5.60e-01   1.059037217e+01   1.053210790e+01   1.9e-02  7.74  
6   7.5e+00  9.7e-03  1.4e-03  1.31e+00   8.391754486e+00   8.378303406e+00   9.7e-03  8.73  
7   1.7e+00  2.2e-03  1.2e-04  1.59e+00   4.310595588e+00   4.308729907e+00   2.2e-03  9.67  
8   7.9e-01  1.0e-03  5.0e-05  5.43e-01   3.008493215e+00   3.006963480e+00   1.0e-03  10.66 
9   2.0e-01  2.6e-04  7.4e-06  6.71e-01   1.335654970e+00   1.335098443e+00   2.6e-04  11.62 
10  6.7e-02  8.7e-05  1.6e-06  6.74e-01   5.232985585e-01   5.230545577e-01   8.7e-05  12.59 
11  1.5e-02  1.9e-05  1.5e-07  1.09e+00   1.330739609e-01   1.330354127e-01   1.9e-05  13.56 
12  5.6e-03  7.2e-06  2.4e-08  1.32e+00   4.754312101e-02   

Obtained the block structure in 1.215149169 seconds. The maximal size of blocks is 45.
Assembling the SDP...
There are 5339 affine constraints.
SDP assembling time: 3.22522247 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 5339            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2235            
  Matrix variables       : 1268            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1   

2   2.9e+01  4.1e-01  7.7e-01  4.06e+00   1.229865084e+01   9.242817578e+00   4.1e-01  1.33  
3   6.0e+00  8.5e-02  4.1e-02  1.89e+00   9.205638894e+00   9.029934241e+00   8.5e-02  1.62  
4   1.9e+00  2.7e-02  8.3e-03  1.02e+00   4.685291910e+00   4.610005255e+00   2.7e-02  2.00  
5   5.3e-01  7.5e-03  1.3e-03  1.15e+00   2.345520021e+00   2.324064224e+00   7.5e-03  2.23  
6   1.4e-01  2.0e-03  2.1e-04  4.15e-01   8.440710469e-01   8.355959156e-01   2.0e-03  2.45  
7   1.6e-02  2.3e-04  6.9e-06  1.07e+00   1.184925381e-01   1.178315260e-01   2.3e-04  2.67  
8   3.0e-03  4.2e-05  3.1e-07  1.56e+00   1.836906432e-02   1.834504526e-02   4.2e-05  2.89  
9   4.2e-04  5.9e-06  1.0e-08  1.76e+00   1.497649459e-03   1.497428347e-03   5.9e-06  3.11  
10  9.4e-07  1.3e-08  9.5e-13  1.11e+00   3.307064722e-06   3.308101227e-06   1.3e-08  3.32  
11  5.4e-09  3.4e-10  4.5e-16  1.00e+00   2.084224896e-08   2.084885538e-08   8.0e-11  3.54  
Optimizer terminated. Time: 3.59    

SDP solving time: 4.15

In [7]:
pred=1
for T in 8:15
    println("T=",T)
    Y=observation(T+pred,ts3,1)
    parameter_estimation2(Y[1:T], 1)
end

T=8
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[5, 3, 2]
[14, 4, 32]
-----------------------------------------------------------------------------
Obtained the block structure in 2.733787316 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 1653 affine constraints.
SDP assembling time: 3.45489317 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1653            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 49              
  Matrix variables       : 100             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator start

2   4.2e+01  4.6e-01  2.0e-01  5.81e+00   7.252437869e+00   7.467135670e+00   4.6e-01  2.31  
3   8.0e+00  8.8e-02  8.4e-03  1.22e+00   1.038955014e+01   1.043605250e+01   8.8e-02  2.87  
4   2.0e+00  2.2e-02  1.5e-03  1.26e+00   5.279232278e+00   5.287547041e+00   2.2e-02  3.45  
5   4.5e-01  5.0e-03  2.5e-04  1.65e-01   2.257963834e+00   2.259697990e+00   5.0e-03  4.00  
6   8.7e-02  9.6e-04  2.6e-05  8.04e-01   6.037067769e-01   6.038718024e-01   9.6e-04  4.54  
7   8.7e-03  9.6e-05  6.8e-07  9.94e-01   6.459468020e-02   6.463224461e-02   9.6e-05  5.07  
8   1.1e-03  1.2e-05  1.3e-08  2.12e+00   4.777567923e-03   4.781989006e-03   1.2e-05  5.65  
9   2.7e-05  3.0e-07  8.9e-11  1.43e+00   8.510483959e-05   8.513242454e-05   3.0e-07  6.18  
10  1.8e-09  3.0e-11  4.6e-17  1.00e+00   5.407442597e-09   5.409840052e-09   1.9e-11  6.77  
Optimizer terminated. Time: 6.77    

SDP solving time: 6.980709395 seconds.
optimum = 5.4074425974000585e-9
The local solver failed!
sol: [-1.91517199393

SDP assembling time: 0.734489996 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2850            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 67              
  Matrix variables       : 136             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00 

2   8.6e+01  7.0e-01  3.4e-01  8.20e+00   6.157039669e+00   6.728790435e+00   7.0e-01  7.98  
3   9.9e+00  8.1e-02  4.4e-02  2.78e+00   1.619050816e+01   1.596661038e+01   8.1e-02  9.94  
4   3.8e+00  3.1e-02  7.6e-03  1.49e+00   7.038242232e+00   6.989941607e+00   3.1e-02  11.88 
5   9.5e-01  7.7e-03  1.3e-03  -9.78e-02  3.274693382e+00   3.251584256e+00   7.7e-03  13.77 
6   1.4e-01  1.2e-03  8.6e-05  8.67e-01   8.302191513e-01   8.257945601e-01   1.2e-03  15.70 
7   9.8e-03  8.0e-05  8.3e-07  9.75e-01   6.819053912e-02   6.813929666e-02   8.0e-05  17.59 
8   1.5e-03  1.2e-05  1.2e-08  2.12e+00   4.111066695e-03   4.114530179e-03   1.2e-05  19.51 
9   6.8e-06  5.5e-08  3.5e-12  1.29e+00   2.107592896e-05   2.108967200e-05   5.5e-08  21.41 
10  2.1e-08  1.7e-10  5.7e-16  1.00e+00   6.457850378e-08   6.462051491e-08   1.7e-10  23.33 
Optimizer terminated. Time: 23.34   

SDP solving time: 23.77773038 seconds.
optimum = 6.457850377841143e-8
Found a locally optimal solution by Ipopt, giv

Assembling the SDP...
There are 4371 affine constraints.
SDP assembling time: 1.522838221 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4371            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 85              
  Matrix variables       : 172             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number           

2   1.3e+02  8.2e-01  4.5e-01  1.04e+01   5.499263762e+00   6.425198217e+00   8.2e-01  63.37 
3   2.1e+01  1.4e-01  1.2e-01  4.89e+00   2.064761253e+01   2.001007020e+01   1.4e-01  69.42 
4   6.1e+00  3.9e-02  1.2e-02  2.79e+00   8.673850009e+00   8.599584684e+00   3.9e-02  74.78 
5   1.8e+00  1.2e-02  2.1e-03  7.83e-01   4.880612302e+00   4.855192969e+00   1.2e-02  80.39 
6   3.7e-01  2.4e-03  1.9e-04  5.64e-01   1.870092000e+00   1.865196908e+00   2.4e-03  86.45 
7   5.2e-02  3.4e-04  1.4e-06  6.26e-01   2.656448910e-01   2.658828004e-01   3.4e-04  92.32 
8   9.5e-03  6.1e-05  7.8e-08  1.36e+00   5.230908970e-02   5.234309674e-02   6.1e-05  97.54 
9   1.3e-03  8.6e-06  7.7e-09  2.13e+00   4.344985550e-03   4.346939329e-03   8.6e-06  102.79
10  2.9e-05  1.9e-07  2.7e-11  1.25e+00   7.709757374e-05   7.713097255e-05   1.9e-07  107.83
11  3.0e-07  2.0e-09  2.8e-14  1.00e+00   8.316619682e-07   8.320102932e-07   2.0e-09  112.92
Optimizer terminated. Time: 113.62  

SDP solving time: 116.